In [26]:
import math
from pylab import *
import matplotlib.pyplot as plt

In [2]:
#Enumeration - mass ordering
massOrdering = ['NMO', 'IMO']
enumMassOrdering = enumerate(massOrdering)

#Enumeration - progenitor
progenitor = ['ONeMg', 'Fe']
enumProgenitor = enumerate(progenitor)

#Enumeration of matter type and flavor of neutrino
matterType = ['nu', 'nubar']
flavor = ['e', 'mu', 'tau']

#Prompting user for input of luminosities and mean energies
L_e = float(raw_input("What is the luminosity of electron neutrinos?: "))
L_anti_e = float(raw_input("What is the luminosity of electron antineutrinos?: "))
L_mu = float(raw_input("What is the luminosity of mu neutrinos?: "))
L_anti_mu = float(raw_input("What is the luminosity of mu antineutrinos?: "))
L_tau = float(raw_input("What is the luminosity of tau neutrinos?: "))
L_anti_tau = float(raw_input("What is the luminosity of tau antineutrinos?: "))
meanE_e = float(raw_input("What is the mean energy of electron neutrinos?: "))
meanE_anti_e = float(raw_input("What is the mean energy of electron antineutrinos?: "))
meanE_mu = float(raw_input("What is the mean energy of mu neutrinos?: "))
meanE_anti_mu = float(raw_input("What is the mean energy of mu antineutrinos?: "))
meanE_tau = float(raw_input("What is the mean energy of tau neutrinos?: "))
meanE_anti_tau = float(raw_input("What is the mean energy of tau antineutrinos?: "))

L_array = [[L_e, L_anti_e],
      [L_mu, L_anti_mu],
      [L_tau, L_anti_tau]]

meanE_array = [[meanE_e, meanE_anti_e],
      [meanE_mu, meanE_anti_mu],
      [meanE_tau, meanE_anti_tau]]

alpha_array = [[3, 3],
      [2, 2],
      [2, 2]]

#Creating a dictionary that is enumerated    
alpha = {}
L = {}
meanE = {}
for x, pair_item in enumerate(flavor):
    for y, item in enumerate(matterType):
        alpha[(item,pair_item)] = alpha_array[x][y]
        L[(item,pair_item)] = L_array[x][y]
        meanE[(item,pair_item)] = meanE_array[x][y]

What is the luminosity of electron neutrinos?: 324
What is the luminosity of electron antineutrinos?: 324123
What is the luminosity of mu neutrinos?: 4324
What is the luminosity of mu antineutrinos?: 3423
What is the luminosity of tau neutrinos?: 657
What is the luminosity of tau antineutrinos?: 43
What is the mean energy of electron neutrinos?: 765
What is the mean energy of electron antineutrinos?: 345
What is the mean energy of mu neutrinos?: 12312
What is the mean energy of mu antineutrinos?: 6545
What is the mean energy of tau neutrinos?: 76456
What is the mean energy of tau antineutrinos?: 132


In [ ]:
#Prompts user input

#distance between supernova and detector on Earth
dist = float(raw_input("What is the distance between the supernova and Earth? (kpc): "))

#progenitor type, prompt user to enter either 'ONeMg' or 'Fe', still need to make case insensitive 
p = raw_input("What is the progenitor type? Please enter either 'ONeMg' or 'Fe': ")

#compactness, prompt user to enter either 2.5 or 5 
comp = float(raw_input("What is the compactness? Please enter either 2.5 or 5: "))

#accretion timescale - units? 
timeAcc = float(raw_input("What is the accretion timescale? "))

#stiffness parameter prompt 
stiff = float(raw_input("What is the stiffness parameter? "))

#prompt user to enter either NMO or IMO, still need to make case insensitive 
mass_ordering = raw_input("What is the mass ordering? Please enter either NMO or IMO: ")

In [3]:
#Factorial definition
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)

In [4]:
#Float range
def frange(start, stop, step = 0.0002):
    while start < stop:
        yield start
        start += step

In [47]:
#Functions

#Ratio of mean energy and luminosity
ratio_e = (L[('nu', 'e')] / meanE[('nu', 'e')])
ratio_anti_e = (L[('nubar', 'e')] / meanE[('nubar', 'e')])
ratio_mu = (L[('nu', 'mu')] / meanE[('nu', 'mu')])
ratio_anti_mu = (L[('nubar', 'mu')] / meanE[('nubar', 'mu')])
ratio_tau = (L[('nu', 'tau')] / meanE[('nu', 'tau')])
ratio_anti_tau = (L[('nubar', 'tau')] / meanE[('nubar', 'tau')])

#Numerator
num_e = (alpha[('nu', 'e')] + 1) ** (alpha[('nu', 'e')] + 1)
num_anti_e = (alpha[('nubar', 'e')] + 1) ** (alpha[('nubar', 'e')] + 1)
num_mu = (alpha[('nu', 'mu')] + 1) ** (alpha[('nu', 'mu')] + 1)
num_anti_mu = (alpha[('nubar', 'mu')] + 1) ** (alpha[('nubar', 'mu')] + 1)
num_tau = (alpha[('nu', 'tau')] + 1) ** (alpha[('nu', 'tau')] + 1)
num_anti_tau = (alpha[('nubar', 'tau')] + 1) ** (alpha[('nubar', 'tau')] + 1)

#Denominator
den_e = (meanE[('nu', 'e')]) * factorial(alpha[('nu', 'e')] + 1)
den_anti_e = (meanE[('nubar', 'e')]) * factorial(alpha[('nubar', 'e')] + 1)
den_mu = (meanE[('nu', 'mu')]) * factorial(alpha[('nu', 'mu')] + 1)
den_anti_mu = (meanE[('nubar', 'mu')]) * factorial(alpha[('nubar', 'mu')] + 1)
den_tau = (meanE[('nu', 'tau')]) * factorial(alpha[('nu', 'tau')] + 1)
den_anti_tau = (meanE[('nubar', 'tau')]) * factorial(alpha[('nubar', 'tau')] + 1)

#Fraction
frac_e = (num_e / den_e)
frac_anti_e = (num_anti_e / den_anti_e)
frac_mu = (num_mu / den_mu)
frac_anti_mu = (num_anti_mu / den_anti_mu)
frac_tau = (num_tau / den_tau)
frac_anti_tau = (num_anti_tau / den_anti_tau)

#File-output for energies of neutrinos (tab delimited) - will output from for loop
#Can change start, stop, and step later - first testing to see if for loop works and writes a text file
for t in range(1, 20, 1):
    #Functions go here
    file = open("data/data_%d.txt" % t, "w+")
    for E in frange(0, 0.1002, 0.0002): 
        flux_e = t*(ratio_e) * (frac_e) * ((E / meanE[('nu', 'e')]) ** (alpha[('nu', 'e')])) * math.exp(-((alpha[('nu', 'e')] + 1) * E) / meanE[('nu', 'e')])
        flux_anti_e = t*(ratio_anti_e) * (frac_anti_e) * ((E / meanE[('nubar', 'e')]) ** (alpha[('nubar', 'e')])) * math.exp(-((alpha[('nubar', 'e')] + 1) * E) / meanE[('nubar', 'e')])
        flux_mu = t*(ratio_mu) * (frac_mu) * ((E / meanE[('nu', 'mu')]) ** (alpha[('nu', 'mu')])) * math.exp(-((alpha[('nu', 'mu')] + 1) * E) / meanE[('nu', 'mu')])
        flux_anti_mu = t*(ratio_anti_mu) * (frac_anti_mu) * ((E / meanE[('nubar', 'mu')]) ** (alpha[('nubar', 'mu')])) * math.exp(-((alpha[('nubar', 'mu')] + 1) * E) / meanE[('nubar', 'mu')])
        flux_tau = t*(ratio_tau) * (frac_tau) * ((E / meanE[('nu', 'tau')]) ** (alpha[('nu', 'tau')])) * math.exp(-((alpha[('nu', 'tau')] + 1) * E) / meanE[('nu', 'tau')])
        flux_anti_tau = t*(ratio_anti_tau) * (frac_anti_tau) * ((E / meanE[('nubar', 'tau')]) ** (alpha[('nubar', 'tau')])) * math.exp(-((alpha[('nubar', 'tau')] + 1) * E) / meanE[('nubar', 'tau')])
        print(str(E)+ '\t' + str(flux_e) + '\t' + str(flux_anti_e) + '\t' + str(flux_mu) + '\t' + str(flux_anti_mu) + '\t' + str(flux_tau) + '\t' + str(flux_anti_tau))
        stringData = str(E)+ '\t' + str(flux_e) + '\t' + str(flux_anti_e) + '\t' + str(flux_mu) + '\t' + str(flux_anti_mu) + '\t' + str(flux_tau) + '\t' + str(flux_anti_tau)
        file = open("data/data_%d.txt" % t, "a")
        file.write(stringData + '\n')
        file.close
    file = open("data/data_%d.txt" %t)
    lines = file.readlines()
    E = []
    flux_e = []
    flux_anti_e = []
    flux_mu = []
    flux_anti_mu = []
    flux_tau = []
    flux_anti_tau = []
    for line in lines:
        E.append(line.split()[0])
    for line in lines:
        flux_e.append(line.split()[1])
    for line in lines:
        flux_anti_e.append(line.split()[2])   
    for line in lines:
        flux_mu.append(line.split()[3])
    for line in lines:
        flux_anti_mu.append(line.split()[4])
    for line in lines:
        flux_tau.append(line.split()[5])
    for line in lines:
        flux_anti_tau.append(line.split()[6])
    file.close()
    plt.figure()
    plt.plot(E, flux_e, 'pink', lw = 2, label="Electron neutrino")
    plt.plot(E, flux_anti_e, 'mediumpurple', lw = 2, label="Electron antineutrino")
    plt.plot(E, flux_mu, 'lightskyblue', lw = 2, label="Mu neutrino")
    plt.plot(E, flux_anti_mu, 'silver', lw = 2, label="Mu antineutrino")
    plt.plot(E, flux_tau, 'gold', lw = 2, label="Tau neutrino")
    plt.plot(E, flux_anti_tau, 'red', lw = 2, label="Tau antineutrino")
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title('Neutrino Spectra_%d' %t)
    plt.xlabel("Energy")
    plt.ylabel("Flux")
    plt.grid()
    plt.savefig('spectra/Neutrino Spectra_%d.png' %t, bbox_inches='tight')
    #plt.show()

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	1.0552517738550604e-22	5.658900764538363e-18	3.387222627850399e-20	3.3576912650878563e-19	3.460920148731195e-24	2.549430996780988e-14
0.0004	8.442005362594965e-22	4.527110113971878e-17	1.3548889851124295e-19	1.3430763829112514e-18	1.384368048628441e-23	1.0197677633938451e-13
0.0006000000000000001	2.849173830346055e-21	1.5278961205138753e-16	3.0485000679405815e-19	3.0219215845215872e-18	3.114828084969908e-23	2.29446703821682e-13
0.0008	6.7535901649053004e-21	3.621671294981824e-16	5.419555412227918e-19	5.372304546654013e-18	5.537472107601466e-23	4.079033971279813e-13
0.001	1.3190591996740319e-20	7.073560345533585e-16	8.46805491893287e-19	8.394225084622787e-18	8.65230010022706e-23	6.373461609778821e-13
0.0012000000000000001	2.2793319134204084e-20	1.2223083933666142e-15	1.219399848901388e-18	1.2087683013742206e-17	1.2459312046550635e-22	9.17774300097305e-13
0.0014	3.6194908737206355e-20	1.9409759571659152e-15	1.659738602342939e-18	1.6452678149326578e-17	1

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	2.1105035477101208e-22	1.1317801529076726e-17	6.774445255700798e-20	6.715382530175713e-19	6.92184029746239e-24	5.098861993561976e-14
0.0004	1.688401072518993e-21	9.054220227943756e-17	2.709777970224859e-19	2.686152765822503e-18	2.768736097256882e-23	2.0395355267876901e-13
0.0006000000000000001	5.69834766069211e-21	3.0557922410277507e-16	6.097000135881163e-19	6.0438431690431745e-18	6.229656169939816e-23	4.58893407643364e-13
0.0008	1.3507180329810601e-20	7.243342589963648e-16	1.0839110824455835e-18	1.0744609093308026e-17	1.1074944215202931e-22	8.158067942559626e-13
0.001	2.6381183993480638e-20	1.414712069106717e-15	1.693610983786574e-18	1.6788450169245574e-17	1.730460020045412e-22	1.2746923219557641e-12
0.0012000000000000001	4.558663826840817e-20	2.4446167867332284e-15	2.438799697802776e-18	2.4175366027484413e-17	2.491862409310127e-22	1.83554860019461e-12
0.0014	7.238981747441271e-20	3.8819519143318304e-15	3.319477204685878e-18	3.2905356298653156e-17	3.

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	3.165755321565181e-22	1.6976702293615088e-17	1.0161667883551196e-19	1.007307379526357e-18	1.0382760446193584e-23	7.648292990342965e-14
0.0004	2.5326016087784893e-21	1.3581330341915635e-16	4.0646669553372884e-19	4.029229148733755e-18	4.153104145885322e-23	3.0593032901815354e-13
0.0006000000000000001	8.547521491038164e-21	4.583688361541627e-16	9.145500203821743e-19	9.065764753564762e-18	9.344484254909725e-23	6.883401114650461e-13
0.0008	2.02607704947159e-20	1.0865013884945473e-15	1.6258666236683751e-18	1.611691363996204e-17	1.6612416322804396e-22	1.223710191383944e-12
0.001	3.957177599022096e-20	2.1220681036600757e-15	2.5404164756798605e-18	2.5182675253868365e-17	2.595690030068118e-22	1.9120384829336463e-12
0.0012000000000000001	6.837995740261224e-20	3.666925180099843e-15	3.658199546704163e-18	3.6263049041226624e-17	3.7377936139651903e-22	2.753322900291916e-12
0.0014	1.0858472621161906e-19	5.822927871497746e-15	4.979215807028817e-18	4.935803444797974e-1

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	4.2210070954202416e-22	2.2635603058153452e-17	1.3548890511401595e-19	1.3430765060351425e-18	1.384368059492478e-23	1.0197723987123953e-13
0.0004	3.376802145037986e-21	1.8108440455887512e-16	5.419555940449718e-19	5.372305531645006e-18	5.537472194513764e-23	4.0790710535753803e-13
0.0006000000000000001	1.139669532138422e-20	6.111584482055501e-16	1.2194000271762326e-18	1.2087686338086349e-17	1.2459312339879633e-22	9.17786815286728e-13
0.0008	2.7014360659621202e-20	1.4486685179927296e-15	2.167822164891167e-18	2.1489218186616053e-17	2.2149888430405863e-22	1.6316135885119252e-12
0.001	5.2762367986961277e-20	2.829424138213434e-15	3.387221967573148e-18	3.357690033849115e-17	3.460920040090824e-22	2.5493846439115283e-12
0.0012000000000000001	9.117327653681633e-20	4.889233573466457e-15	4.877599395605552e-18	4.8350732054968826e-17	4.983724818620254e-22	3.67109720038922e-12
0.0014	1.4477963494882542e-19	7.763903828663661e-15	6.638954409371756e-18	6.581071259730631e-

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	5.276258869275302e-22	2.829450382269182e-17	1.6936113139251995e-19	1.678845632543928e-18	1.7304600743655975e-23	1.274715498390494e-13
0.0004	4.221002681297483e-21	2.2635550569859393e-16	6.774444925562148e-19	6.715381914556257e-18	6.921840243142204e-23	5.098838816969226e-13
0.0006000000000000001	1.4245869151730277e-20	7.639480602569378e-16	1.5242500339702908e-18	1.510960792260794e-17	1.557414042484954e-22	1.1472335191084099e-12
0.0008	3.37679508245265e-20	1.8108356474909124e-15	2.709777706113959e-18	2.6861522733270065e-17	2.7687360538007328e-22	2.0395169856399063e-12
0.001	6.595295998370161e-20	3.536780172766793e-15	4.234027459466435e-18	4.1971125423113936e-17	4.32615005011353e-22	3.18673080488941e-12
0.0012000000000000001	1.1396659567102043e-19	6.111541966833072e-15	6.09699924450694e-18	6.043841506871103e-17	6.229656023275317e-22	4.588871500486525e-12
0.0014	1.809745436860318e-19	9.704879785829579e-15	8.298693011714696e-18	8.22633907466329e-17	8.47925

...

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	1.5828776607825907e-21	8.488351146807544e-17	5.080833941775599e-19	5.0365368976317835e-18	5.191380223096793e-23	3.824146495171482e-13
0.0004	1.2663008043892449e-20	6.790665170957816e-16	2.032333477668645e-18	2.0146145743668767e-17	2.0765520729426612e-22	1.5296516450907675e-12
0.0006000000000000001	4.273760745519082e-20	2.291844180770813e-15	4.572750101910873e-18	4.5328823767823804e-17	4.672242127454863e-22	3.4417005573252297e-12
0.0008	1.0130385247357952e-19	5.432506942472736e-15	8.129333118341877e-18	8.058456819981018e-17	8.306208161402199e-22	6.118550956919719e-12
0.001	1.9785887995110482e-19	1.0610340518300379e-14	1.2702082378399307e-17	1.259133762693418e-16	1.297845015034059e-21	9.560192414668229e-12
0.0012000000000000001	3.4189978701306123e-19	1.8334625900499213e-14	1.829099773352082e-17	1.813152452061331e-16	1.8688968069825954e-21	1.3766614501459575e-11
0.0014	5.429236310580955e-19	2.911463935748873e-14	2.4896079035144087e-17	2.4679017223989864e

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	1.6884028381680966e-21	9.054241223261381e-17	5.419556204560638e-19	5.37230602414057e-18	5.537472237969912e-23	4.079089594849581e-13
0.0004	1.3507208580151944e-20	7.243376182355005e-16	2.1678223761798873e-18	2.1489222126580023e-17	2.2149888778055054e-22	1.6316284214301521e-12
0.0006000000000000001	4.558678128553688e-20	2.4446337928222005e-15	4.8776001087049304e-18	4.8350745352345396e-17	4.983724935951853e-22	3.671147261146912e-12
0.0008	1.0805744263848481e-19	5.7946740719709185e-15	8.671288659564668e-18	8.595687274646421e-17	8.859955372162345e-22	6.526454354047701e-12
0.001	2.110494719478451e-19	1.1317696552853737e-14	1.3548887870292592e-17	1.343076013539646e-16	1.3843680160363297e-21	1.0197538575646113e-11
0.0012000000000000001	3.6469310614726534e-19	1.9556934293865828e-14	1.9510397582422207e-17	1.934029282198753e-16	1.9934899274481016e-21	1.468438880155688e-11
0.0014	5.791185397953017e-19	3.1055615314654643e-14	2.6555817637487025e-17	2.63242850389225

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	1.793928015553603e-21	9.620131299715217e-17	5.758278467345678e-19	5.708075150649355e-18	5.883564252843031e-23	4.33403269452768e-13
0.0004	1.4351409116411442e-20	7.696087193752192e-16	2.30331127469113e-18	2.2832298509491275e-17	2.353425682668349e-22	1.7336051977695367e-12
0.0006000000000000001	4.8435955115882935e-20	2.5974234048735883e-15	5.1824501154989884e-18	5.137266693686699e-17	5.2952077444488435e-22	3.900593964968594e-12
0.0008	1.148110328033901e-19	6.156841201469101e-15	9.21324420078746e-18	9.132917729311822e-17	9.41370258292249e-22	6.934357751175682e-12
0.001	2.242400639445854e-19	1.2025052587407096e-14	1.4395693362185874e-17	1.4270182643858738e-16	1.4708910170386e-21	1.0834884736623995e-11
0.0012000000000000001	3.8748642528146944e-19	2.0779242687232442e-14	2.0729797431323593e-17	2.0549061123361753e-16	2.1180830479136075e-21	1.5602163101654188e-11
0.0014	6.153134485325082e-19	3.2996591271820566e-14	2.821555623982996e-17	2.796955285385518e-16	2.

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	1.8994531929391086e-21	1.0186021376169054e-16	6.0970007301307185e-19	6.043844277158142e-18	6.22965626771615e-23	4.588975794205779e-13
0.0004	1.5195609652670938e-20	8.148798205149381e-16	2.4388001732023736e-18	2.4175374892402528e-17	2.4918624875311934e-22	1.8355819741089214e-12
0.0006000000000000001	5.1285128946228985e-20	2.750213016924976e-15	5.487300122293047e-18	5.439458852138858e-17	5.606690552945835e-22	4.1300406687902764e-12
0.0008	1.215646229682954e-19	6.519008330967284e-15	9.755199742010252e-18	9.670148183977225e-17	9.967449793682639e-22	7.342261148303663e-12
0.001	2.3743065594132576e-19	1.2732408621960457e-14	1.5242498854079167e-17	1.510960515232102e-16	1.5574140180408706e-21	1.1472230897601878e-11
0.0012000000000000001	4.102797444156735e-19	2.2001551080599057e-14	2.1949197280224983e-17	2.1757829424735976e-16	2.2426761683791142e-21	1.6519937401751494e-11
0.0014	6.515083572697145e-19	3.493756722898648e-14	2.987529484217291e-17	2.961482066878784

0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0002	2.0049783703246147e-21	1.075191145262289e-16	6.435722992915757e-19	6.379613403666927e-18	6.575748282589271e-23	4.843918893883877e-13
0.0004	1.6039810188930433e-20	8.601509216546569e-16	2.5742890717136164e-18	2.5518451275313777e-17	2.6302992923940374e-22	1.937558750448306e-12
0.0006000000000000001	5.4134302776575047e-20	2.903002628976363e-15	5.792150129087105e-18	5.741651010591016e-17	5.918173361442827e-22	4.359487372611958e-12
0.0008	1.283182131332007e-19	6.881175460465466e-15	1.0297155283233043e-17	1.0207378638642625e-16	1.0521197004442786e-21	7.750164545431645e-12
0.001	2.506212479380661e-19	1.3439764656513814e-14	1.6089304345972454e-17	1.5949027660783296e-16	1.6439370190431415e-21	1.2109577058579759e-11
0.0012000000000000001	4.330730635498776e-19	2.322385947396567e-14	2.316859712912637e-17	2.2966597726110193e-16	2.367269288844621e-21	1.74377117018488e-11
0.0014	6.877032660069208e-19	3.68785431861524e-14	3.1535033444515837e-17	3.1260088483720497e-16